✓ Image found at: /Users/tk20/Desktop/APCOMP215/data/sample_imgs/trial/a.png


/Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/frontend-react/python/test_cv_analysis_001
No case history file found for case test_cv_analysis_001, using default.
No conversation history file found for case test_cv_analysis_001, using default.
No demographics file found for case test_cv_analysis_001, using default.


Initialized API Manager for case: test_cv_analysis_001


    Running CV analysis on: /Users/tk20/Desktop/APCOMP215/data/sample_imgs/trial/a.png


Running CV analysis...


    ✓ CV analysis complete
    Metrics: {'compactness_index': 7.793015138374817, 'color_stats_lab': {'mean_L': 97.50898065172332, 'mean_A': 139.1335732283343, 'mean_B': 149.33476140207583, 'std_L': 23.84394686179632, 'std_A': 3.762945670075602, 'std_B': 2.9665262673640056}, 'area_cm2': 20.29215199863236, 'area_cm2_uncorrected': 19.35286141826923, 'tilt_correction_factor': 0.9537116329294973}



CV Analysis Results:
{
  "metrics": {
    "compactness_index": 7.793015138374817,
    "color_stats_lab": {
      "mean_L": 97.50898065172332,
      "mean_A": 139.1335732283343,
      "mean_B": 149.33476140207583,
      "std_L": 23.84394686179632,
      "std_A": 3.762945670075602,
      "std_B": 2.9665262673640056
    },
    "area_cm2": 20.29215199863236,
    "area_cm2_uncorrected": 19.35286141826923,
    "tilt_correction_factor": 0.9537116329294973
  },
  "bbox": [
    0,
    0,
    1024,
    558
  ],
  "coin_data": [
    752,
    271,
    78
  ],
  "pixels_per_cm": 81.88976377952756,
  "tilt_correction": 0.9537116329294973
}
